In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/souhu2022/generated_train_data_piece.txt
/kaggle/input/souhu2022/generated_test_data_piece.txt


In [2]:
import pandas as pd
train = pd.read_json('../input/souhu2022/generated_train_data_piece.txt', lines = True)
test = pd.read_json('../input/souhu2022/generated_test_data_piece.txt', lines = True)
train.head()

,id,content,entity,label
0,1,3.新疆棉是全球业界公认的高品质天然纤维原料，较好满足了全球范围内对棉制纺织品服装的刚性消费...,美国,0
1,1,3.新疆棉是全球业界公认的高品质天然纤维原料，较好满足了全球范围内对棉制纺织品服装的刚性消费...,中国,0
2,2,显然，与其指望德约在罗兰-加洛斯击败纳达尔，不如把希望寄托在墨尔本，寄托在梅德韦杰夫身上。去...,德约,-1
3,2,显然，与其指望德约在罗兰-加洛斯击败纳达尔，不如把希望寄托在墨尔本，寄托在梅德韦杰夫身上。去...,梅德韦杰夫,1
4,2,显然，与其指望德约在罗兰-加洛斯击败纳达尔，不如把希望寄托在墨尔本，寄托在梅德韦杰夫身上。去...,澳网,0


In [3]:
test.head()

,id,content,entity
0,5000001,然而作为同龄的选手，王涵也不想把所有的压力都压在施廷懋一人的肩上，“当然我也可以给予她一些帮...,施廷懋
1,5000001,然而作为同龄的选手，王涵也不想把所有的压力都压在施廷懋一人的肩上，“当然我也可以给予她一些帮...,王涵
2,5000002,◆诺贝尔奖是如何评选的据诺贝尔奖委员会官网介绍，诺贝尔奖由瑞典发明家、企业家阿尔弗雷德·诺贝...,阿尔弗雷德·诺贝尔
3,5000002,◆诺贝尔奖是如何评选的据诺贝尔奖委员会官网介绍，诺贝尔奖由瑞典发明家、企业家阿尔弗雷德·诺贝...,诺贝尔奖委员会
4,5000003,上半场比赛中威少这边的进攻状态依旧不是太好，只不过对比威少以往的表现，这场比赛威少的进攻也算...,威少


In [4]:
train['text'] = '目标:' + train['entity'] + '[SEP]' + train['content']
train.head()

,id,content,entity,label,text
0,1,3.新疆棉是全球业界公认的高品质天然纤维原料，较好满足了全球范围内对棉制纺织品服装的刚性消费...,美国,0,目标:美国[SEP]3.新疆棉是全球业界公认的高品质天然纤维原料，较好满足了全球范围内对棉制...
1,1,3.新疆棉是全球业界公认的高品质天然纤维原料，较好满足了全球范围内对棉制纺织品服装的刚性消费...,中国,0,目标:中国[SEP]3.新疆棉是全球业界公认的高品质天然纤维原料，较好满足了全球范围内对棉制...
2,2,显然，与其指望德约在罗兰-加洛斯击败纳达尔，不如把希望寄托在墨尔本，寄托在梅德韦杰夫身上。去...,德约,-1,目标:德约[SEP]显然，与其指望德约在罗兰-加洛斯击败纳达尔，不如把希望寄托在墨尔本，寄托...
3,2,显然，与其指望德约在罗兰-加洛斯击败纳达尔，不如把希望寄托在墨尔本，寄托在梅德韦杰夫身上。去...,梅德韦杰夫,1,目标:梅德韦杰夫[SEP]显然，与其指望德约在罗兰-加洛斯击败纳达尔，不如把希望寄托在墨尔本...
4,2,显然，与其指望德约在罗兰-加洛斯击败纳达尔，不如把希望寄托在墨尔本，寄托在梅德韦杰夫身上。去...,澳网,0,目标:澳网[SEP]显然，与其指望德约在罗兰-加洛斯击败纳达尔，不如把希望寄托在墨尔本，寄托...


In [5]:
import os
import random
import warnings
from collections import defaultdict
import sys
sys.path.append('./nezha')
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, AutoModel, AutoConfig, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
warnings.filterwarnings('ignore')

In [6]:
class Config:
    apex = False
    seed = 42
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_workers = 0
    
    model_type = 'roberta'
    model_name = 'hfl/chinese-roberta-wwm-ext'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    text_col = 'text'
    target_col = 'label'
    target_size = 5
    max_len = 400
    batch_size = 8
    n_fold = 2
    trn_folds = [0, 1]
    
    epochs = 4
    lr = 2e-5
    eps = 1e-6
    betas = (0.9, 0.999)
    num_warmup_steps = 0.03
    T_0 = 2
    min_lr = 1e-6
    
    weight_decay = 0.01
    gradient_accumulation_steps = 1
    max_grad_norm = 1.0
    
    save_prefix = model_name.split('/')[-1]

CFG = Config()

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
def seed_torch(seed=42):
    """
    设置随机种子
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_torch(CFG.seed) # 固定运行环境

In [8]:
class SentimentDataset(Dataset):
    def __init__(self, texts, targets, labels, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        '''
        item is the index of data,迭代取第item条数据
        '''
        text = str(self.texts[item])
        target = str(self.targets[item])
        label = self.labels[item]
        
        encoding = self.tokenizer.encode_plus(text,
                                             target,
                                             truncation = 'only_first', #True/only_first，将每个文本序列截断到模型可以接受的最大长度
                                            add_special_tokens = True,
                                            max_length = self.max_len,
                                              return_token_type_ids = True,
                                              pad_to_max_length = True,
                                              return_attention_mask = True,
                                              return_tensors = 'pt', #返回张量
                                             )
        
        return {'texts': text,
               'input_ids': encoding['input_ids'].flatten(), #对应于文本序列中每个token的索引（在vocab中的索引）
               'attention_mask': encoding['attention_mask'].flatten(), #对应于注意力机制的计算，各元素的值为0或1，如果当前token被mask或者是只是用来作为填充的元素，那么其不需要进行注意力机制的计算，其值为0；
               'token_type_ids': encoding['token_type_ids'].flatten(),#对应于不同的文本序列，例如在NSP（BERT及某些语言模型中的“Next Sentence Prediction”）任务中需要输入两个文本序列。
               'labels': torch.tensor(label, dtype = torch.long)
               }

In [9]:
# def create_data_loader(df, tokenzier, max_len, batch_size, is_shuffle = False):
#     ds = SentimentDataset(texts = df['text'].values,
#                          targets = df['entity'].values,
#                          labels = df['label'].values,
#                          tokenizer = tokenizer,
#                          max_len = max_len
#                          )
#     return DataLoader(ds,
#                      batch_size = batch_size,
#                      shuffle = is_shuffle,
#                      pin_memory = True, #设置pin_memory=True，则意味着生成的Tensor数据最开始是属于内存中的锁页内存，这样将内存的Tensor转义到GPU的显存就会更快一些；配置好，设置为锁页内存；默认为False，即不锁页内存
#                      )

def create_data_loader(df, tokenizer, max_len, batch_size, is_shuffle=False):
    ds = SentimentDataset(
        texts=df['text'].values,
        targets=df['entity'].values,
        labels=df['label'].values,
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=is_shuffle,
        pin_memory=True,
        # num_workers=4  # windows多线程
    )

In [10]:
class CLSModel(nn.Module):
    def __init__(self, model_name, n_classes):
        super(ClSModel, self).__init__()
        config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name, config = config)
        self.linear = nn.Linear(config.hidden_size, n_classes)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.model(input_ids, attention_mask, token_type_ids)
        last_hidden_state = outputs[0]
        cls_embeddings = last_hidden_state[:, 0] # pooler out
        logits = self.linear(cls_embeddings)
        return logits
 

In [11]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, epoch):
    model = model.train()
    losses = []
    predictions = []
    real_values = []
    tqdm_bar = tqdm(data_loader, total = len(data_loader), desc = f'training epoch"\t {epoch}')
    for batch in tqdm_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        targets = batch['labels'].to(device)
        
        outputs = model(input_ids = input_ids,
                       attention_mask = attention_mask,
                       token_type_ids = token_type_ids
                       )
        loss = loss_fn(outputs, targets)
        _, preds = torch.max(outputs, dim = 1) #返回每一行中最大值的那个元素，且返回其索引(代表类别)
        
        losses.append(loss.item()) # !!!!!!!!!
        loss.backward() #不是优化器，也可以反向传播嘛？
        
        tqdm_bar.set_postfix_str(f'running training loss:{loss.item():.4f}')
        nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5.0) #梯度裁剪。在BP过程中会产生梯度消失（就是偏导无限接近0，导致长时记忆无法更新），可以设定阈值，当梯度小于阈值时，更新的梯度为阈值，（梯度裁剪解决的是梯度消失或爆炸的问题）
        optimizer.step() #对模型参数进行更新
        scheduler.step() #对lr进行调整，如果scheduler.step()是放在batch里面，指的是经过step_size次的迭代，学习率改变一次
        optimizer.zero_grad()
        
        predictions.extend(preds) #.extend() 在列表末尾一次性追加另一个序列的多个值
        real_values.extend(targets)
    predictions = torch.stack(predictions).cpu() # torch.stack()增加新的维度进行堆叠
    real_values = torch.stack(real_values).cpu()
    return f1_score(predictions, real_values, average = 'macro'), np.mean(losses)

        

In [12]:
def eval_model(model, data_loader, loss_fn, device, epoch):
    model = model.eval() #验证预测模式
    losses = []
    predictions = []
    real_values = []
    with torch.no_grad(): #上下文管理器，被该语句 wrap 起来的部分将不会track 梯度，执行计算时，该计算不会被反向传播记录
        for batch in tqdm(data_loader, total = len(data_loader), desc = f'evaluating:\t {epoch}'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            targets = batch['labels'].to(device)
            
            outputs = model(input_ids = input_ids,
                           attention_mask = attention_mask,
                           token_type_ids = token_type_ids)
            _, preds = torch.max(outputs, dim = 1)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())
            predictions.extend(preds)
            real_values.extend(targets)
    predictions = torch.stack(predictions).cpu()
    real_values = torch.stack(real_values).cpu()
    return f1_score(predictions, real_values, average = 'macro'), np.mean(losses)
        

In [13]:
class FocalLoss(nn.Module):
    def __init__(self, alpha = 1, gamma = 2, reduce = True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce
    
    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss()(inputs, targets)
        pt = torch.exp(-BCE_loss) #变为指数
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss
        

In [14]:
def train_cv():
    best_scores = []
    skf = StratifiedKFold(n_splits = CFG.n_fold, shuffle = True, random_state = CFG.seed)
    for n, (train_index, val_index) in enumerate(skf.split(train, train[CFG.target_col])):
        if n in CFG.trn_folds:
            print(f'Fold {n + 1} / {CFG.n_fold}')
            print('***' * 10)
            
            df_train = train.loc[train_index]
            df_val = train.loc[val_index]
            train_data_loader = create_data_loader(df_train, CFG.tokenizer, CFG.max_len, CFG.batch_size, True)
            val_data_loader = create_data_loader(df_val, CFG.tokenizer, CFG.max_len, CFG.batch_size, False)
            model = LastHiddenModel(CFG.model_name, CFG.target_size)
            model = model.to(CFG.device)
            
            no_decay = ['bias', 'LayerNorm.weight']
            
            param_optimizer = list(model.named_parameters())
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                 'weight_decay_rate': CFG.weight_decay},
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                 'weight_decay_rate': 0.0}
            ]
            
            optimizer = AdamW(optimizer_grouped_parameters, lr = CFG.lr, weight_decay = CFG.weight_decay)
            total_steps = len(train_data_loader) * CFG.epochs
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                       num_warmup_steps = CFG.num_warmup_steps,
                                                       num_training_steps = total_steps)
            loss_fn = FocalLoss().to(CFG.device)
            
            history = defaultdict(list) #记录10轮loss和acc
            best_accuracy = 0
            
            for epoch in range(CFG.epochs):
                train_acc, train_loss = train_epoch(model,
                                                   train_data_loader,
                                                   loss_fn,
                                                   optimizer,
                                                   CFG.device,
                                                   scheduler,
                                                   epoch,)
                print(f'\n Train loss {train_loss} accuracy {train_acc}')
                
                val_acc, val_loss = eval_model(model,
                                              val_data_loader,
                                              loss_fn, 
                                              CFG.device,
                                              epoch)
                print(f'\n Val loss {val_loss} accuracy{val_acc}')
                
                history['train_acc'].append(train_acc)
                history['train_loss'].append(train_loss)
                history['val_acc'].append(val_acc)
                history['val_loss'].append(val_loss)
                
                if val_acc > best_accuracy:
                    torch.save(model.state_dict(), f'models/{CFG.save_prefix}_best_model_fold{n}.bin')
                    best_accuracy = val_acc
            best_scores.append(best_accuracy)
    print(best_scores)
    print(np.mean(best_scores))


In [15]:
def get_predictions(model, data_loader):
    model = model.eval()
    predictions_probs = []
    with torch.no_grad():
        for d in tqdm(data_loader,):
            input_ids = d["input_ids"].to(CFG.device)
            attention_mask = d["attention_mask"].to(CFG.device)
            token_type_ids = d["token_type_ids"].to(CFG.device)
            
            outputs = model(input_ids = input_ids,
                           attention_mask = attention_mask,
                           token_type_ids = token_type_ids)
            probs = outputs.softmax(1)
            prediction_probs.extend(probs)
    predicitons_probs = torch.stack(predicitons_probs).cpu()
    return prediction_probs


In [16]:
class LastHiddenModel(nn.Module):
    def __init__(self, model_name, n_classes):
        super().__init__()

        config = AutoConfig.from_pretrained(model_name)

        self.model = AutoModel.from_pretrained(model_name, config=config)
        self.linear = nn.Linear(config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.model(input_ids, attention_mask, token_type_ids)# last_hidden_state和pooler out
        last_hidden_state = outputs[0] # 所有字符最后一层hidden state # 32 400 768 ，但是PAD PAD
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        logits = self.linear(mean_embeddings)
        return logits

In [17]:
train_cv()

Fold 1 / 2
******************************


Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
training epoch"	 0:   0%|          | 0/1438 [00:10<?, ?it/s]


IndexError: Target -1 is out of bounds.